## Нейронная сеть Lenet5.

Lenet5 - это одна из первых сверточных нейронных сетей и она отражает характерные для сверточных нейронных сетей набор элементов - сверточные слои, пуллинг слои и полносвязные слои на конце нейронной сети. Данная архитектура послужила основой для многих современных архитектур сверточных нейронных сетей.

In [1]:
from keras.datasets import mnist
from keras.utils import np_utils
from keras import utils
from keras.utils import to_categorical
import keras.utils as np_utils
from keras.models import Sequential
from keras import models, layers
import keras

In [2]:
# загрузка тренировочных и тестовых данных
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# конвертация чисел из uint8 в float32
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# нормализация данных [0, 1]
x_train /= 255
x_test /= 255

# трансформация лейблов в one-hot encoding
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

# изменение размерности массива в 4D массив
x_train = x_train.reshape(x_train.shape[0], 28,28,1)
x_test = x_test.reshape(x_test.shape[0], 28,28,1)

In [3]:
# инициализация пустой модели
model = Sequential()

# первый сверточный слой
# Conv2D - с изображениями, Conv1D - с текстом

# model.add(layers.Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation='tanh', input_shape=(28,28,1), padding="same"))
# kernel_size=(5, 5) - окошко 5х5
# strides=(1, 1) - шаг окошка (проходится без пропусков)
# activation='tanh' - функция активации
# input_shape=(28,28,1) - входное изображение
# padding="same" - дополнительный слой 0-пикселей по краям изображения - чтобы сохранить размерность

# второй пуллинговый слой
model.add(layers.AveragePooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))

# AveragePooling2D - усредненные значения
# pool_size=(2, 2) - размер окошка
# strides=(1, 1) - шаг
# padding='valid' - без слоя пикселей по краям - размер изображения уменьшается


# третий сверточный слой
model.add(layers.Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid'))

# четвертый пуллинговый слой
model.add(layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

# strides=(2, 2) - шаг 2, через клетку - размер изображения уменьшается

# пятый полносвязный слой
model.add(layers.Conv2D(120, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid'))

# сглаживание CNN выхода чтобы можно было его присоединить к полносвязному слою
model.add(layers.Flatten())

# шестой полносвязный слой
model.add(layers.Dense(84, activation='tanh'))

# выходной слой с функцией активации softmax
model.add(layers.Dense(10, activation='softmax'))


# компилияция модели
model.compile(loss=keras.losses.categorical_crossentropy, 
              optimizer='SGD', 
              metrics=["accuracy"])


In [4]:
hist = model.fit(x=x_train,y=y_train, epochs=3, batch_size=128, validation_data=(x_test, y_test), verbose=1)

test_score = model.evaluate(x_test, y_test)
print("Test loss {:.4f}, accuracy {:.2f}%".format(test_score[0], test_score[1] * 100))

Epoch 1/3
469/469 [==============================] - 24s 49ms/step - loss: 0.7546 - accuracy: 0.8158 - val_loss: 0.3960 - val_accuracy: 0.8953
Epoch 2/3
469/469 [==============================] - 22s 46ms/step - loss: 0.3636 - accuracy: 0.8994 - val_loss: 0.3103 - val_accuracy: 0.9131
Epoch 3/3
313/313 [==============================] - 3s 8ms/step - loss: 0.2665 - accuracy: 0.9271
Test loss 0.2665, accuracy 92.71%


In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 average_pooling2d (AverageP  (None, 27, 27, 1)        0         
 ooling2D)                                                       
                                                                 
 conv2d (Conv2D)             (None, 23, 23, 16)        416       
                                                                 
 average_pooling2d_1 (Averag  (None, 11, 11, 16)       0         
 ePooling2D)                                                     
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 120)         48120     
                                                                 
 flatten (Flatten)           (None, 5880)              0         
                                                                 
 dense (Dense)               (None, 84)                4